<span type="title">Hibernate Basic</span> | <span type="update">2018-10-24</span> - Version <span type="version">1.0</span>
    
    
<span type="intro"><p class="card-text">本章主要介绍 Hibernate 的基础使用。</p></span>

# Object/Relation Map

Hibernate 是一个对象持久化框架，广义上的持久化指的是 OOP 对象和 数据库行列之间的交互，包括 增加、删除、更改、查找（CRUD）。Hibernate 特有一个叫做“加载”的概念，这是指根据 OID，将一个对象从数据库加载到内存中的操作。其中 OID 指的是，对象标识，OO 对象的属性对应数据库的列，OO 的每个对象对应数据库的行，这种映射称之为 ORM(Object/Relation Map)。为了满足对象和 OO 之间的对应，我们所需要一个指定关联，这个关联叫做 Object Identifier。

ORM 不仅仅需要有 OID 的行对应，还需要有对象的列对应，列对应属性。在 DBUtils or JdbcTemplate 中，我们约定好了一种简单的映射规则，其中属性全小写作为数据库列名标识，这样就完成了对应。但对于实际情况，一个对象可能包含非基本类型的其它类型属性，这时候就需要使用多张表完成此对象的映射，在这种情况下，我们常常需要提供一个 .xml 文件完成这种属性和列的手动映射。

ORM 的架构分为三层，上层是业务逻辑，这里包含了域模型，其中包含了 ORM 需要处理的一些问题：对象、属性、关联、继承和多态。中间层是 ORM 提供的 API，以及其具体实现。Java 有官方的 API，叫做 JPA，Hibernate 是 JPA 的实现的一个超集，支持 xml 和注解两种方式定义域模型和映射关系。下层是数据库层，这里负责维护关系数据模型，包括表、字段、索引、主键和外键。业务逻辑层和数据库层之间通过 xml 或者注解联系起来，完成类和数据库表之间的映射，以支持 CRUD 的操作。

ORM 的业务逻辑域模型被封装成 Data Access Object 对象，负责进行 CRUD 的操作。DAO 可以由 JDBC，DBUtiles，JdbcTemplate 来实现，也可以由 Hibernate，MyBatis 来实现。前者称之为工具，按照约定来进行简单映射，后者称之为框架，其中 Hibernate 使用 xml 文件或者注解进行域模型和数据模型的全自动关联，MyBaits 使用 xml 文件或者 Mapper 接口配合 SQL 进行域模型和数据模型的手动关联。

# Hello World by Hibernate 

为了使用 Hibernate 创建 “Hello， World”，最基本的，我们需要有域模型内的 Java 持久化对象，此外还要指定此对象的 OID，属性和数据表列的对应，除此之外，我们还要指定数据库的登录和配置信息，这样，Hibernate 就可以自动在 Java 对象和 数据库表之间建立联系。在整个过程中，我们不需要编写 SQL，预先提供数据表，整个配置完全自动化。其中映射文件，一般以 `.hbm.xml` 结尾，Hibernate 配置文件，以 `hibernate.cfg.xml` 作为名称。

之后，我们就可以通过在 Data Access Object 中通过操作 Hibernate 提供的 API，或者 JPA 标准的 API 完成数据 CRUD 了。

## Hiberante 配置文件

一个基本的 Hibernate.cfg.xml 文件的配置如下, 其中主要包含 session-factory 定义的创建一个连接所需要的配置信息：

- connection 类别的有 username 用户名、password 密码、url 数据库及地址、driver_class 数据库驱动、数据库方言、isolation 数据库隔离级别。
- debug 类别的有 show_sql 显示SQL语句，format_sql 格式化SQL语句显示，hbm2ddl.auto 表的一些操作：总是新建、更新现有、用完即删
- c3p0 数据源的可选配置有 max_size 最大连接, min_size 最小连接, acquire_increment 每次递增，max_statements 最大会话，timeout 检测超时关闭标准，idle_test_period 多久检测一次超时
- jdbc 类别的有：fatch/batch_size 每次查询多少数据、每次批处理执行多少数据（均对 Oracle 有效）
- current_session_class 标签： 基于什么样的方式获取 session 类
- mapping 标签：定义域模型和数据模型关联的配置文件。

```xml
<?xml version='1.0' encoding='utf-8'?>
<!DOCTYPE hibernate-configuration PUBLIC "-//Hibernate/Hibernate Configuration DTD//EN" "http://www.hibernate.org/dtd/hibernate-configuration-3.0.dtd">
<hibernate-configuration>
  <session-factory>
    <!--提供基本的 hibernate 对象和数据库交互时的参数，比如用户名、密码、地址、驱动
    是否在控制台显示SQL，是否自动创建表-->
    <property name="connection.username">corkine</property>
    <property name="connection.password">mi960032</property>
    <property name="connection.driver_class">com.mysql.jdbc.Driver</property>
    <property name="connection.url">jdbc:mysql://localhost:3306/orm</property>
    <!--影响 refresh 能否获取最新的数据，隔离级别四种：1 读未提交 2 读已提交 4 可重复读 8 序列化-->
    <property name="connection.isolation">2</property>

    <property name="show_sql">true</property>
    <property name="format_sql">true</property>
    <!--常见的hbm2ddl.auto的选择： update（更新表，不存在则创建，如果表结构不同，则更新表结构）、create（每次均创建新表）、
    create-drop（用完即删），validate（如果表结构不同或者不存在表，则抛出异常）-->
    <property name="hbm2ddl.auto">update</property>


    <!--配置 datasource 数据源-->
    <property name="c3p0.max_size">1</property>
    <property name="c3p0.min_size">1</property>
    <property name="c3p0.acquire_increment">1</property>
    <property name="c3p0.max_statements">100</property>
    <!--一个链接多长时间没有用，超时设置为 timeout、idel_test 为多长时间检测一次-->
    <property name="c3p0.timeout">1000</property>
    <property name="c3p0.idle_test_period">1000</property>

    <!--配置管理 Session 的方式: 基于 thread-->
    <property name="current_session_context_class">thread</property>

    <!--配置 oracle 的 fetch-size, 每次取回的 sql 行数 batch_size 每次批量更新的 sql 语句数
    注意，此配置仅对 oracle 数据库有效。-->
    <property name="hibernate.jdbc.fetch_size">100</property>
    <property name="hibernate.jdbc.batch_size">30</property>
    <!--使用路径，而不是包名。提供对象映射设置文件-->
    <mapping resource="com/mazhangjing/hibernate/News.hbm.xml" />
    <mapping resource="com/mazhangjing/prohib/Worker.hbm.xml" />
    <mapping resource="com/mazhangjing/hql/Department.hbm.xml" />
  </session-factory>
</hibernate-configuration>
```

## 数据对象及 Hibernate 数据映射

一个持久化 Java 类主要信息，注意，这里 Date 为 java.util.Date，此处的 Bolb 为 java.sql.Blob。省略了 get/set 方法以及无参构造器。

```java
public class News {
    private Integer id;
    private String title;
    private String author;
    private Date date;
    private String desc;
    private String content;
    private Blob image;
    ...
}
```

POJO必须是非 final 类，因为 Hibernate 会使用代理。要提供一个无参构造器，以供通过反射进行对象的获取。此外，最好提供一个标识属性，以映射数据库表的主键字段。并且，最好重写 equals 和 hashCode 方法以进行唯一性隔离。

通过代理和反射创建对象后的调用过程 :

- 通过 .cfg.xml（或者 .properties 文件，或者指定路径下的xml或者prop文件）的设置连接数据库
- 通过提供 .hbm.xml 的对象映射文件来映射 Bean 属性和 SQL表、列、属性
- 通过 setter 和 getter 进行持久化对象和数据库字段的交互
- 通过 ORM API 提供SQL语句进行查询、写入表、更新表

一个非常简单的类-数据库映射文件：

```xml
<hibernate-mapping package="com.mazhangjing.hibernate">
    <class name="News" table="Daily" select-before-update="false" dynamic-update="true">
        <id name="id" type="java.lang.Integer" unsaved-value="none">
            <column name="ID" />
            <generator class="native"/>
        </id>
        <property name="title" type="string" column="TITLE" update="false" unique="true" index="news_index"/>
        <property name="author" type="java.lang.String" column="AUTHOR" index="news_index" />
        <property name="date" type="time" >
            <column name="DATE" />
        </property>
        <property name="desc" type="string" formula="(select concat(author, ':', title) from daily n where n.id = id)" />
        <property name="content" type="string">
            <column name="content" sql-type="mediumtext" />
        </property>
        <property name="image" type="blob">
            <column name="image" sql-type="mediumblob" />
        </property>
    </class>
</hibernate-mapping>
```

**class**

在这个映射文件中，需要提供 hibernate-mapping 标签，其中可以包含数个 class 类标签，package 属性可以定义这些 class 共有的 package，这样，class 中只用指定类名即可，而不用指定全类名。

class 标签需要提供 Java 持久化类的一些映射信息，包括此类对应的数据库表，因此，需要提供 name、table 这两个属性。class 标签内的其余属性也拿过来设置类和表映射的信息，比如动态更新，谨慎更新等。

- dynamic-update 只更新修改的字段，而不更新没有包含的字段
- dynamic-insert 同上
- select-before-update 更新前查询，以确定是否需要更新，影响性能

**id**

id 标签规定了 Object identifier，需要在 name 指定 Java 属性，在 type 指定转换类型，此转换类型将对接两边的类型，进行自动转换，此标签其余属性可定义主键相关设置。推荐使用代理主键（没有任何业务含义的值），整型（节约查询资源）。

- unsaved-value saveOrUpdate 时判断为 save 而不是 update

generator 主键增长方式：
1. increment hibernate 递增，首先确定此表中最大值，然后max(id)+1，并发时容易出现问题
2. identity 由底层数据库负责生成标识符，必须为 long,int 或者 short 类型的数据
3. sequence 由 sequence 标识符生成器从数据库的指定 param 定义的序列获得一个唯一标识号，再作为主键值（Oracle,DB）
4. hilo Hibernate 由高低算法生成标识符，适合所有数据库系统。（hibernate自己维护一个表和列，读取后立马修改next_high，这样就没有并发问题）
5. native 自动根据数据库能力，选择 identity、sequence或者hilo

**property**

property 标签规定了除了 OID 的其余属性（基本类型）的映射，同样的，需要指定 name（Java属性）、type（转换类型）、column（对应的列的名称）。此外，可以使用 update 限定禁止更新（允许新建和删除），unique 指定唯一，index 指定建立索引。此标签可提供 column 子标签，此子标签 name 属性等同于父标签的 column 属性。为了精确完成映射，可以不选择 type 进行自动转换，而在 column 标签指定 sql-type 指定 SQL 端的标签类型（此类型因数据库不同而不同）。

对于 type，如果不提供，则通过反射获取并直接使用 pojo 类型，并且转换为 hibernate 类型。此处可以为 java 类型或者 hibernate 类型。如果设置为 java 类型，则自动转换为 hibernate 类型。

对于 formula，持久化类的有些属性的值必须在运行时通过计算得到，这些pojo属性并不和表中单独一列对应，使用 formula 处理这种映射对应关系
注意，formula 本质是一个子查询，使用括号括起来注意内查询和外查询的对应关系。有 formula 就不需要提供 column 了。

此外，scale 指定小数的位数，对于 double, float, decimal 等类型有效。length 指定字段长度。

**时间日期类型映射**

其中 Java 内置的时间日期为 util.Date, Calender JDBC 扩展了 sql.Date, Time, TimeStamp， 这三种分别对应 SQL 的 DATE, TIME, TIMESTAMP
(Timestamp包含时间和日期信息)。因为 util.Date 是 sql 三个扩展类型的父类，因此推荐使用 util.Date 以包含所有可能的数据库时间日期类型种类。

在 hbm.xml 中，使用 type 属性将 pojo 的 util.Date 映射为 hibernate 类型 time、date、timestamp。这样的话，数据库就会使用 time\date\timestamp 类型。

注意，必须在 pojo 设置 util.Date 类型，如果设置为 sql.Date 类型，而 xml 设置 time 类型，则会转换出错。

**大对象类型映射**

对于Java 而言， 使用 `String` 表示长字符串， `byte[]` 表示图片或者文件二进制， `sql.Clob sql.Blob` 表示 CLOB（字符串大对象） BLOB（二进制大对象）。但是对于 mysql，其不支持 CLOB， 使用 `text mediumtext longtext` 表示长文本。<u>因此，在持久化中，使用 `String` 而不是 `sql.CLOB` 表示长文本（更方便），使用 `sql.BLOB` 而不是 `byte[]` 表示二进制数据</u>（`byte[]` 需要繁琐的转换进行保存和读写，而 Blob 直接可以通过 hibernate.createXXX 获取 io 流）。

由于指定 type 可能导致不恰当的映射，比如 string - mediumtext/text， 使用 sql-type 来精确指定 sql 列的类型。实际使用中，经常是通过文件的方式保存大二进制数据（图片等），而在 sql 数据库中保存位置的引用，而不是直接存储在数据库中（大Blob影响性能）。
因此，最常用的 pojo 类型是 String，对应的 mysql 类型（sql-type） 是 `mediumtext, largetext`。而 `BLOB` 和 `byte[]` 则很少用。

API 中，使用 Hibernate.getLobCreator(Session).createBlob 来加载一个流，指定流长度，来创建 Bolb 对象。使用 getBinaryStream 直接从 Bolb 中获得输出流写入到 FileOutputStream 中即可。

```java
@Test public void setBlob() throws IOException, SQLException {
    InputStream stream = new FileInputStream("cat.jpg");
    Blob blob = Hibernate.getLobCreator(session).createBlob(stream,stream.available());
    News news = new News(1,"aa","AA",new Date());
    news.setImage(blob);
    session.save(news);
    //注意，不能在 save 前关闭 stream，否则会报错。其次，pojo 使用 blob 类更加方便（方便set和get，方便xml映射hibernate blob
    // sql-type:medium-blob）。
}
@Test public void readBlod() throws IOException, SQLException {
    News news = session.get(News.class,1);
    Blob blob = news.getImage();
    FileOutputStream fos = new FileOutputStream("data_in_sql.jpg");
    InputStream in = blob.getBinaryStream();
    int length; byte[] buffer = new byte[1024];
    while ((length = in.read(buffer)) != -1) {
        fos.write(buffer);
    }
}
```


## Hibernate ORM API

一个基本的 ORM API 操纵流程为, 获取配置文件，根据配置文件创建 SF 对象，根据 SF 创建 Session，此处的 Session 类似于 JDBC 的 Connection，负责管理每次数据库调用。创建完 Session 后，获取 Transaction，之后执行操作，基本的操作API为 get，获取指定主键的指定类型的对象，save 保存一个对象到对应映射的数据库表中。然后提交事务、关闭 Session、关闭 SF。

```java
//关联cfg.xml文件
Configuration configuration = new Configuration().configure("hibernate.cfg.xml");
//创建sessionFactory对象
SessionFactory sessionFactory = configuration.buildSessionFactory();
//创建Session
Session session = sessionFactory.openSession();
//开启事务
Transaction transaction = session.beginTransaction();
//执行保存操作
/*News news = session.get(News.class,1);
System.out.println(news);*/
News news = new News(1,"Java","Marvin",new Date());
session.save(news);
//提交事务
transaction.commit();
//关闭Session
session.close();
//关闭SessionFactory
sessionFactory.close();
```

**Configuration API：**

读取 hibernate 配置文件，cfg.xml（调用.configure()方法） 或者 hibernate.properties（不调用.configure()的话）

**SessionFactory API:**

创建Session的工厂，一旦建造完毕，则赋予特定配置信息，比如对象映射、各项数据验证信息。由于创建很消耗资源，因此一个
项目只有一个。从此工厂创建 session。在新版本中的 SF 使用如下方法开启(通过 MS build Mdata，然后进一步由 Mdata 构建 SF，其中 MS 需要使用 SSR, SSR 通过 SSRB 通过传入配置文件调用 configure 来 build)：

```java
sessionFactory = new MetadataSources(
    new StandardServiceRegistryBuilder()
        .configure("hibernate.cfg.xml")
        .build())
    .buildMetadata()
    .buildSessionFactory();
```

**Session API:**

Session 从 Factory 获取数据库信息，然后连接 JDBC，操作数据库。Session 是应用程序和数据库操作的核心，单线程存在。其生命周期很短，在flush之前，所有持久层数据存放在此对象中。相当于 JDBC 的 Connection。
- 获取：get() load()
- 更新：update() save() saveOrUpdate() delete()
- 事务：beginTransaction()
- 管理：isOpen() flush() clear() evict() close()
- Session 提供数据库操纵方法。Session 类似于 JDBC 的 connection 对象。

**Transaction API：**

事务概念，表示一次原子操作。推荐读、写都使用事务。如果不通过事务，不能写入数据到数据库中。提供 commit() rollback() wasCommitted()
等方法

## Hiberante 测试用例

Hibernate 的一个测试用例如下：

```java
public class NewsTest {
    private SessionFactory sessionFactory;
    //实际使用不能作为成员变量，有并发的问题
    private Session session;
    private Transaction transaction;
    @Before
    public void init() {
        Configuration configuration = new Configuration().configure();
        sessionFactory = configuration.buildSessionFactory();
        session = sessionFactory.openSession();
        transaction = session.beginTransaction();
    }
    @After
    public void destroy() {
        transaction.commit();
        session.close();
        sessionFactory.close();
    }
    @Test
    public void go() {...}
}
```

注意，在实际情况下，session 和 transaction 不能作为成员变量，有并发问题。此外，实际情况下的 SF 通常和线程绑定，采用如下方式获得 session：

```java
public class HibernateUtils {

    private static HibernateUtils instance = new HibernateUtils();
    private SessionFactory sessionFactory;
    private Session session;

    public static HibernateUtils getInstance() { return instance;}

    private HibernateUtils() {}

    public SessionFactory getSessionFactory() {
        if (sessionFactory == null) {
            Configuration configuration = new Configuration().configure();
            sessionFactory = configuration.buildSessionFactory();
        }
        return sessionFactory;
    }

    public Session getSession() {
        return getSessionFactory().getCurrentSession();
    }
}
```

`getCurrentSession` 返回和线程绑定的 Session，这种方式和 Service 层解除了耦合。当使用 thread 管理的时候，不用关 session，只用提交事务，session 会自动被处理。注意，使用这种方法必须开启基于线程的 session 获取方式。



# Session 和对象操纵

## Session 缓存

Session 是 Hibernate 最常用的一个接口，其主要用于数据库操纵，比如CRUDL。其具有一个缓存，保存着一些 Java POJO。缓存的意义在于，多次相同的查询，可以直接从缓存获得对象，而不是重复向数据库发送请求：

当进行 get 查询的时候，如果找到后，则向引用变量赋值，同时保存引用到 Session 缓存中。这意味着，当引用失效后，Java对象不会被回收，因此下一次 get 会先从 session 缓存中找到对象，进行复用。这也就是为什么要写 id、hashCode 和 equal 方法的原因了。只要 Session 中的缓存没有结束生命周期，那么这个对象将不会被 gc。

```java
News news = session.get(News.class,1);
News news2 = session.get(News.class,1); //实际只执行了一次查询语句。
```

注意这种“高效”是如何完成的：每进行一次查询，获得的对象有两个引用，其一交给用户，其二保存在缓存中，以备下次查询。这和 CPU 多级缓存类似。

缓存操作有三：`flush() reflesh() clear()` 其中 flush 缓存向数据库更新， reflesh 用于数据库向缓存更新， clear 用于清空缓存

**flush**

flush 使数据库和 Session 缓存中对象状态保持一致，如下的语句执行了 select 和 update 语句，更新了对象。
     
```java
News news = session.get(News.class,3);
news.setTitle("Again");
```

注意，在调用的时候，并没有真正的进行数据库的 update，在 flush 后才发送到数据库，但此时尚未提交事务，当最后事务提交后，数据才会被更改。

flush 一般不需要我们手动调用，在 trans.commit() 执行过程中已经被调用了。如下：

 【何时调用】在 trans.commit() 方法中，先调用 session 的 flush() 方法，再提交事务
 
 【如何更新】flush 会自动编写并执行 update 语句，但是不会提交事务，因此在事务结束前不能看到数据更改
 
 【一些特例】这意味着，只有调用 commit 才会进行 SQL 语句操作同步更改，并且提交事务。但是，在未提交事务或者显式调用 session.flush() 之前，可能也会进行 flush() 操作：
     - 执行 HQL, QBC 查询，会先进行 flush 操作，得到最新的数据库记录
     - 如果记录的 ID 是底层数据库自增产生的，在调用 save() 方法后，会自动发送 insert 语句（在 commit 的 flush 之前）获取 id
     - 需要注意，发生这些特例的时候，Hibernate 自动 flush，自然丧失了缓存的优势
     
如果一个全新的对象在自增模式下进行 save，则立即 flush，如果是原先存在的对象，则自动在 commit 执行 flush，不用手动 save:

```java
News news = new News(3,"Hello Java","Marvin",new Date());
session.save(news); //自增下强制 flush 获取 OID 

News news2 = session.get(News.class,1);
news2.setTitle("Hi");
session.save(news2); //多次一举
```

**refresh**

refresh 用来更新 POJO，使其和数据库当前记录保持一致：

```java
News news = session.get(News.class,3);
System.out.println(news);
//如果想要此值得最新字段：
session.refresh(news); //可以看到 hibernate 执行了 select 语句。
//但是这里可能得到的不是最新的数据，这是 MySQL 的事务隔离级别设置问题。更改设置即可。
```

**clear**

clear 会清空 session 中的缓存：

```java
News news = session.get(News.class,3);
session.clear(); 
//清除缓存后，会重新执行查询语句（对于此情况，执行了两条查询）
News news2 = session.get(News.class,3);
```

## Hibernate 对象的状态

根据是否具有 OID，是否存在于缓存、数据表中，区分了四种不同的对象，其中持久化对象和游离对象具有 OID，属于在编，一个在工作，一个在休假。而临时对象和删除对象没有 OID，要么处于入职前的状态，要么处于离职后的状态。

- 持久化对象 - 有 oid，存在于 缓存和数据表中（工作的正式成员）
- 游离对象 - 有 oid，存在于数据表中，但是不存在缓存中（非工作正式成员）
- 临时对象 - 无 oid，不存在于 缓存和数据表中（非正式员工，尚未参与工作）
- 删除对象 - 无 oid，不存在于 缓存和数据库中。

其工作流程如下：
     
起点：一个 POJO 通过 new 变成一个临时对象，此对象没有 oid 和 数据库对应，不存在缓存中。或者通过 get 和 load 直接变成持久化对象。

- 临时对象：一个临时对象通过 save/persist/saveOrUpdate/merge 变成持久化对象，此时具有了 oid，具有缓存和数据库记录。如果没有引用，且尚未变成持久化对象，则进行垃圾回收。

- 持久化对象：一个持久化状态对象通过 evict/close/clear 变成游离状态，此时的 oid 依旧存在，数据库记录也存在，但是不存在于缓存中。或者直接通过 delete 进行删除，编程删除对象。

- 游离对象：一个游离对象通过 update、saveOrUpdate、merge 变成 持久化对象，通过 delete 变成删除对象。

- 删除对象：删除对象直接被 gc。

**临时状态的 save, persist 方法**

save 方法将临时对象变为持久化对象。

<u>save 的 OID 确认问题：</u> 因为临时对象没有 oid，因此 save 方法会为临时对象分配 oid。（通过 insert 语句返回的 id 作为 oid，此操作不在 commit 的 flush 中进行）。在 save 之前设置 id 无效，因为 id 需要执行 insert 和 数据库协商（自增）来确定。在 save 之后，也就是持久化对象的时候，id 不可修改，否则无法进行数据库和缓存对应。

```java
News news = new News(null,"Learn Python","Wiz",new Date());
news.setId(233333);
session.save(news);
//news.setId(34444); 
```

persist 从原始 Class 变为 临时对象，然后变为 持久化对象。persist 和 save 方法类似，都可以将 POJO 从游离状态变成持久状态。

和 save 方法的区别是： 在 persist 之前不能 setId，区别于 save 直接忽略设置 id 的语句，persist 直接抛出异常。

```java
News news = new News(null,"Learn Python","Wiz",new Date());
//news.setId(666);
session.persist(news);
```

**get, load 方法获得持久化状态**

get 从原始 Class 直接变为持久化对象:

```java
News news = session.get(News.class,3);
System.out.println(news);
```

【当存在记录时】load 和 get 的功能差不多，但是 load 并没有马上执行查询，除非调用了此对象的方法，比如 get 或者 toString 此外。如果在调用引用进行操作前关闭了 session 或者 transaction，如果存在数据，则 get 方法正常使用（因为整个对象都被加载到了本地），如果不存在，get 方法则返回 null。但是 load 可能抛出懒加载异常（代理对象在需要时不能通过 session 填充数据）
 
【不存在记录时】如果在使用前未关闭 session 和 transaction，并且数据库中不存在对应的记录，那么 get 返回 null，而 load 抛出异常。

```java
News news = session.load(News.class,3);
System.out.println(news.getClass().getSimpleName());
```

**游离状态的 update 方法**

如果更新一个持久化对象， 不需要使用 update（默认在事务 commit 会调用 session 的 flush，而 flush 会自动将缓存中的持久化对象保存到数据库中）

如果更新一个游离状态对象（一般是之前的 session 关闭，而新建了一个 session 而造成的游离），则需要使用 update 进行加入缓存（入职）的操作。

注意：如果新开一个 session 和 transaction，无论要更新的游离对象和数据表的记录是否一致，都会发送 update 语句。但是，如果在一个 session 和 transaction 中，如果缓存和记录表一致，调用 update 不发送。换句话说，对于游离对象，update 不知道是否数据变更，因此必须强制发送 update，而对于持久化对象，因为 update 前我们可以比较数据是否变更，因此如果不变更则不发送。

可以通过在 hbm.xml 中设置 `select-before-update = true` 来在更新前强制 select， 如果数据有变更，则 update，否则不 update。这种方式可以避免触发器协同工作问题，但是影响效率（如果对持久化对象操作比对游离对象操作少，且需要和触发器协同工作的情况下可用）

【查无此人】注意：如果在数据表中不存在对象记录，则 update 会抛异常（即 update 只用于有 oid 的游离和持久化对象，不适用于临时对象）

【真假冲突】注意：如果使用 update 更新一个对象（来自于游离对象），而 session 中也存在一个 oid 相同的对象（来自于持久化对象），则更新抛出异常。

```java
//对于持久化对象，不需要 update
//News news = session.get(News.class,3);
//news.setAuthor("Marvin");

//对于持久化对象，如果 update，并且没有更改（相比缓存），则不触发 update
//News news = session.get(News.class,3);
//session.update(news);

//对于游离状态（有 oid，但是没有在缓存中，则需要手动 update 来更新游离对象（入职））
News news = session.get(News.class,3);
transaction.commit();
session.close();

session = sessionFactory.openSession();
transaction = session.beginTransaction();
news.setAuthor("Corkine");
session.update(news);
```

**使用 saveOrUpdate 更新游离或临时对象**

update 不能处理临时对象，saveOrUpdate 如果是临时对象，调用 save，如果是游离对象，则调用 update 通过设置一个数据库具有的 oid 可以将临时对象变成游离对象，或者来自其余 session 的有 oid 的对象也是游离对象。

对于临时对象还是游离对象的判断标准在于是否具有 OID。

即：如果没有 oid，则认为是 临时对象，调用 save。如果有 oid（不论是手动设置还是本身就有），则认为是 游离对象，调用 update。和 update 方法一样，如果不存在对应 oid 的数据库记录，则抛出异常。

但是，有时候可能需要为 临时对象添加一个 OID 标识，在这种情况下，可以单独设置 id 的 unsaved-value 属性，则如果有 oid，且 oid 和此值对应，则认为是游离对象，进行 save 而不是 update。之后此 OID 标识会被清除，更换为新的 OID。

```java
News news = new News(null,"Mars","Marvin",new Date());
news.setId(3); //通过为临时对象设置 oid，现在 news 表示的是一个游离对象(除非 oid 等于 unsaved-value)
session.saveOrUpdate(news);
```

**使用 delete 删除持久化或游离对象**

delete 用于删除具有 oid 的游离对象或者持久化对象。

只要 oid 和数据表中一条记录对应，就会准备执行 delete 操作。但是如果不存在此 oid 对应值，则抛出异常。 

delete 在 commit 和 flush 时执行。通过 `hibernate.use_identifier_rollback` 设置为 true，让 delete 后对象 oid 变为 null。在执行 delete 后(尚未提交前)即改变。

```java
@Test public void delete() {
    //对于持久化对象
    //News news = session.get(News.class,4);
    //session.delete(news);
    //对于游离对象
    //News news1 = new News();
    //news1.setId(5);
    //session.delete(news1);
}
```

